In [81]:
import torch
import torch.nn as nn
import pandas as pd
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader,TensorDataset

### Load csv File

In [82]:


def load_data(file_path):
    df = pd.read_csv("/content/car_purchasing.csv")  # Corrected file path usage
    feature_columns = df.columns[:-1]  # All columns except the last
    target_column = df.columns[-1]  # Last column
    x = df[feature_columns].values  # Feature columns
    y = df[target_column].values  # Target column
    return x, y


In [83]:
device="Cuda" if torch.cuda.is_available() else "cpu"
device

'Cuda'

### Prepare Dataset

In [84]:
def prepare_dataset(x,y):
  scaler=StandardScaler()
  x=scaler.fit_transform(x)
  x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=42)

  x_train =torch.tensor(x_train,dtype=torch.float32)
  y_train=torch.tensor(y_train,dtype=torch.float32).view(-1,1)
  x_test=torch.tensor(x_test,dtype=torch.float32)
  y_test=torch.tensor(y_test,dtype=torch.folat32).view(-1,1)

  train=TensorDataset(x_train,y_train)
  test=TensorDataset(x_test,y_test)

  return train,test



### Define ANN model

In [85]:
class ANN(nn.Module):
  def __init__(self,input_dim): # input dimension
    super(ANN,self). __init__()
    self.flatten=nn.Flatten() # flatten is convert into "multi-dimensional input"
    self.linear1=nn.linear1(input_dim,100) # Fully connected layer 1
    self.linear2=nn.linear2(100,50) # Fully connected layer 2
    self.linear3=nn.linear3(50,1) # Output layer
    self.relu=nn.Relu()  # Activation function

    def forward(self,x):
      x=self.flatten(x)
      x=self.linear1(x)
      x=self.relu(x)
      x=self.linear2(x)
      x=self.relu(x)
      x=self.linear3(x)

      return x


In [86]:
def train_model(model,train,criterion,optimizer,epochs=5):
  for epoch in range(epochs):
    for x_batch,y_batch in train:
      optimizer.zero_grad()
      y_pred=model(x_batch)
      loss=criterion(y_pred,y_batch)
      loss.backward()
      optimizer.step()

    if (epoch+1)%10==0:
      print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


In [87]:
def test_model(model,test,criterion):
  model.eval()
  with torch.no_grad():
    total_loss=0
    for x_batch,y_batch in test:
      y_pred=model(x_batch)
      loss=criterion(pred,y_batch)
      total_loss+=loss.item()
      print(f'Test Loss:{total_loss/len(test):.4f}')

### Save Model

In [94]:
def save_model(model,file_path="/content/sample_data/Car_purchasing ANN_model"):
  model.load_state_dict(torch.load(file_path))
  model.eval()
  print("Model loaded from{file_path}")

### Load The Model

In [98]:
def load_model(model,file_path="/content/sample_data/Car_purchasing ANN_model"):
  model.load_state_dict(torch.load(file_path))
  model.to(device)
  model.eval()
  print("Model loaded successfully!")


### Prediction Function

In [96]:
def predict(model,x_test):
  model.eval()
  with torch.no_grad():
    predictions= model(torch.tensor(x_test,dtype=torch.float32))
  return predictions.numpy()


### Main Function

In [108]:
from google.colab import files
uploaded = files.upload()


In [109]:
import os
print(os.listdir("/content"))  # Lists all files in the '/content' directory


['.config', 'FashMNIST', 'sample_data']


In [111]:
file_path = "/content/Car_purchasing ANN_model.csv"


In [112]:
import os
print(os.getcwd())  # Prints the current working directory


/content


In [119]:
# Execute script without main function
file_path = "/content/Car_purchasing ANN_model.csv"  # Adjust path as needed
batch_size = 64

X, y = load_data(file_path)
train_loader, test_loader, X_test, y_test = prepare_dataset(X, y, batch_size)

input_dim = X.shape[1]
model = ANN(input_dim)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

train_model(model, train_loader, criterion, optimizer)
test_model(model, test_loader, criterion)

save_model(model)

# Load and predict
load_model(model)
predictions = predict(model, X_test)
print("Predictions:", predictions[:10])

FileNotFoundError: [Errno 2] No such file or directory: '/content/car_purchasing.csv'